<a href="https://colab.research.google.com/github/Florian-toll/Hello-Worlds/blob/master/CNNAugen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive

drive.mount('/content/gdrive')
%cd gdrive/MyDrive/Datasets/Augen/0
!ls



Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
[Errno 2] No such file or directory: 'gdrive/MyDrive/Datasets/Augen/0'
/content/gdrive/MyDrive/Datasets/Augen/0
closed-input-0	no_face-input-0  open-input-0


In [14]:
import cv2
import numpy as np
import os
import sys
import tensorflow as tf
import sklearn
from tensorflow import keras
from tensorflow.keras import regularizers
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

from sklearn.model_selection import train_test_split


PATH_TO_DRIVE = "/content/gdrive/My Drive"
# number of learnings should be until the loss converges otherwise we overfit
EPOCHS = 10
IMG_WIDTH = 32
IMG_HEIGHT = 32
NUM_CATEGORIES = 3
# number of images getting used for testing not for training in percent
TEST_SIZE = 0.2
# drop out nodes at random to make the network more robust
DROPOUT = 0.5
dropout_rate = 0.0
NORMALIZE = True
HIDDENLAYERS_SIZE = 128
STEPS_PER_EPOCH = 1
# batchsize = x -> could define one // woundn't use it except if it takes to long
# it is the amount of images in the data set which is getting looked at before an adjustment


def main():
    # Get image arrays and labels for all image files
    images, labels = load_data(os.getcwd())
    print(labels)
    # Split data into training and testing sets
    labels = tf.keras.utils.to_categorical(labels)
    x_train, x_test, y_train, y_test = train_test_split(
        np.array(images), np.array(labels), test_size=TEST_SIZE
    )
    # Get a compiled neural network
    model = KerasClassifier(build_fn = lambda:get_model())
    # Fit model on training data
    #model.fit(x_train, y_train, epochs=EPOCHS)
    
    batch_size = [10, 20, 60, 100]
    epochs = [20]
    dropout_rate = [0.1, 0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
    pool_type =  ['max', 'average']
    param_grid = dict(dropout_rate=dropout_rate)
    grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, cv=3,
                        verbose = 2)
    grid_result = grid.fit(x_train, y_train)
    # summarize results
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))
    # Evaluate neural network performance
    

    model.evaluate(x_test,  y_test, verbose=1)
    """
    %cd ..
    %cd ..
    !ls 
    filename = ("/content/gdrive/MyDrive/Deep_CNN/final")
    !ls
    model.save(filename)
    %cd Datasets/Augen
    """
    # Save model to file
    #if len(sys.argv) == 3:
    #    filename = sys.argv[2]
    #    model.save(filename)
    #    #print(f"Model saved to {filename}.")

def load_data(data_dir,paths = None):
    """
    Load image data from directory `data_dir`.

    Assume `data_dir` has one directory named after each category, numbered
    0 through NUM_CATEGORIES - 1. Inside each category directory will be some
    number of image files.

    Return tuple `(images, labels)`. `images` should be a list of all
    of the images in the data directory, where each image is formatted as a
    numpy ndarray with dimensions IMG_WIDTH x IMG_HEIGHT x 3. `labels` should
    be a list of integer labels, representing the categories for each of the
    corresponding `images`.
    """
    images = list()
    labels = list()

    # get currend directory
    #workingDirectory = os.getcwd()

    # add them together
    #path = workingDirectory + os.sep + data_dir
    path = data_dir
    # loop over all the directories
    for dirpath, dirnames, filenames in os.walk(path):
        print("q")
        if(filenames != ['.DS_Store']):
            # loops over all the files inside the directory
            for curFile in filenames:
                if(curFile != '.DS_Store' and curFile != 'a'):
                    file = dirpath + os.sep + curFile
                    # reads the image
                    image = cv2.imread(file)
                    # rezeises the image
                    resized = cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT),
                                        interpolation=cv2.INTER_AREA)
                    # make the input better for the neural network
                    if(NORMALIZE):
                        resized = resized / 255.0
                    # add to pictures
                    images.append(resized)
                    if paths != None:
                      paths.append(file)
                    # add to labels
                    if (dirpath == PATH_TO_DRIVE+"/Datasets/Augen/3/closed-input-3" or 
                        dirpath == PATH_TO_DRIVE+"/Datasets/Augen/2/closed-input-2" or
                        dirpath == PATH_TO_DRIVE+"/Datasets/Augen/1/closed-input-1" or
                        dirpath == PATH_TO_DRIVE+"/Datasets/Augen/0/closed-input-0"):
                        labels.append(0)
                    if (dirpath == PATH_TO_DRIVE+"/Datasets/Augen/1/no_face-input-1" or
                        dirpath == PATH_TO_DRIVE+"/Datasets/Augen/2/no_face-input-2" or
                        dirpath == PATH_TO_DRIVE+"/Datasets/Augen/3/no_face-input-3" or 
                        dirpath == PATH_TO_DRIVE+"/Datasets/Augen/0/no_face-input-0"):
                        labels.append(1)
                    if (dirpath == PATH_TO_DRIVE+"/Datasets/Augen/3/open-input-3" or
                        dirpath == PATH_TO_DRIVE+"/Datasets/Augen/2/open-input-2" or
                        dirpath == PATH_TO_DRIVE+"/Datasets/Augen/1/open-input-1" or
                        dirpath == PATH_TO_DRIVE+"/Datasets/Augen/0/open-input-0"):
                        labels.append(2)
    return(images, labels)

def get_model():
    """
    Returns a compiled convolutional neural network model. Assume that the
    `input_shape` of the first layer is `(IMG_WIDTH, IMG_HEIGHT, 3)`.
    The output layer should have `NUM_CATEGORIES` units, one for each category.
    """
    INPUT_SHAPE = (IMG_WIDTH, IMG_HEIGHT, 3)

    #makes the network more robust
    data_augmentation = tf.keras.Sequential(
      [
        tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal", 
                                                    input_shape=(IMG_HEIGHT, 
                                                                  IMG_WIDTH,
                                                                  3)),
        tf.keras.layers.experimental.preprocessing.RandomRotation(0.1),
        tf.keras.layers.experimental.preprocessing.RandomZoom(0.1),
      ]
    )

    # decrease the rate of change towards the end
    lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
        0.001,
        decay_steps=STEPS_PER_EPOCH*1000,
        decay_rate=1,
        staircase=False)
    # bias
    #model with several layers
    model = tf.keras.models.Sequential([
        data_augmentation,
        # add the convolution layer -> extract feature such as strong colour difference
        #-> lernt Kanten
        tf.keras.layers.Conv2D(128, (3, 3), activation="relu",
                               input_shape=INPUT_SHAPE),
        # use pooling -> verkleinert das Bild
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        # add another concolutional layer
        tf.keras.layers.Conv2D(256, (3, 3), activation="relu"),
        # pool again
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        #new
        tf.keras.layers.Conv2D(256, (3, 3), activation="relu"),
        # generate a form as an input layer
        tf.keras.layers.Flatten(),
        #refularisation punishes more komplex networks
        tf.keras.layers.Dense(
                HIDDENLAYERS_SIZE,
                kernel_regularizer=regularizers.l2(0.01),
                activation='relu'),
        # dense all are connected -> smaller then the imput layer
        tf.keras.layers.Dense(
                HIDDENLAYERS_SIZE,
                kernel_regularizer=regularizers.l2(0.01),
                activation='relu'),
        # add a dropout
        tf.keras.layers.Dropout(dropout_rate),
        # add the output layer -> softmax gives a probability distribution min loss = 0 max = infinity
        # continues until the end
        tf.keras.layers.Dense(NUM_CATEGORIES, "softmax")
    ])
    # compile the model
    model.compile(
        optimizer=tf.keras.optimizers.Adam(lr_schedule),
        loss="categorical_crossentropy", 
        metrics=["accuracy"]
        #metrics=[tf.keras.metrics.Accuracy]
    )
    return model

if __name__ == "__main__":
  main()

dict_keys(['explained_variance', 'r2', 'max_error', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'adjusted_rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_weighted'])
q
q
q
q
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

ValueError: ignored

In [ ]:
import cv2
import numpy as np
import os
import sys
import tensorflow as tf

from sklearn.model_selection import train_test_split

# number of learnings should be until the loss converges otherwise we overfit
EPOCHS = 12
IMG_WIDTH = 30
IMG_HEIGHT = 30
NUM_CATEGORIES = 3
# number of images getting used for testing not for training in percent
TEST_SIZE = 0.2
# drop out nodes at random to make the network more robust
DROPOUT = 0.9
NORMALIZE = True
HIDDENLAYERS_SIZE = 128
# batchsize = x -> could define one // woundn't use it except if it takes to long
# it is the amount of images in the data set which is getting looked at before an adjustment


def main():
    # Get image arrays and labels for all image files
    images, labels = load_data(os.getcwd())
    print(labels)
    # Split data into training and testing sets
    labels = tf.keras.utils.to_categorical(labels)
    x_train, x_test, y_train, y_test = train_test_split(
        np.array(images), np.array(labels), test_size=TEST_SIZE
    )
    # Get a compiled neural network
    model = get_model()
    # Fit model on training data
    model.fit(x_train, y_train, epochs=EPOCHS)

    # Evaluate neural network performance
    model.evaluate(x_test,  y_test, verbose=1)
    
    %cd ..
    %cd ..
    !ls 
    filename = ("/content/gdrive/MyDrive/Deep_CNN/final")
    !ls
    model.save(filename)
    %cd Datasets/Augen

    # Save model to file
    #if len(sys.argv) == 3:
    #    filename = sys.argv[2]
    #    model.save(filename)
    #    #print(f"Model saved to {filename}.")

def load_data(data_dir):
    """
    Load image data from directory `data_dir`.

    Assume `data_dir` has one directory named after each category, numbered
    0 through NUM_CATEGORIES - 1. Inside each category directory will be some
    number of image files.

    Return tuple `(images, labels)`. `images` should be a list of all
    of the images in the data directory, where each image is formatted as a
    numpy ndarray with dimensions IMG_WIDTH x IMG_HEIGHT x 3. `labels` should
    be a list of integer labels, representing the categories for each of the
    corresponding `images`.
    """
    images = list()
    labels = list()

    # get currend directory
    #workingDirectory = os.getcwd()

    # add them together
    #path = workingDirectory + os.sep + data_dir
    path = data_dir
    # loop over all the directories
    for dirpath, dirnames, filenames in os.walk(path):
        print("q")
        if(filenames != ['.DS_Store']):
            # loops over all the files inside the directory
            for curFile in filenames:
                if(curFile != '.DS_Store' and curFile != 'a'):
                    file = dirpath + os.sep + curFile
                    # reads the image
                    image = cv2.imread(file)
                    # rezeises the image
                    resized = cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT),
                                        interpolation=cv2.INTER_AREA)
                    # make the input better for the neural network
                    if(NORMALIZE):
                        resized = resized / 255.0
                    # add to pictures
                    images.append(resized)
                    # add to labels
                    if (dirpath == "/content/gdrive/My Drive/Datasets/Augen/3/closed-input-3" or 
                        dirpath == "/content/gdrive/My Drive/Datasets/Augen/2/closed-input-2" or
                        dirpath == "/content/gdrive/My Drive/Datasets/Augen/1/closed-input-1" or
                        dirpath == "/content/gdrive/My Drive/Datasets/Augen/0/closed-input-0"):
                        labels.append(0)
                    if (dirpath == "/content/gdrive/My Drive/Datasets/Augen/1/no_face-input-1" or
                        dirpath == "/content/gdrive/My Drive/Datasets/Augen/2/no_face-input-2" or
                        dirpath == "/content/gdrive/My Drive/Datasets/Augen/3/no_face-input-3" or 
                        dirpath == "/content/gdrive/My Drive/Datasets/Augen/0/no_face-input-0"):
                        labels.append(1)
                    if (dirpath == "/content/gdrive/My Drive/Datasets/Augen/3/open-input-3" or
                        dirpath == "/content/gdrive/My Drive/Datasets/Augen/2/open-input-2" or
                        dirpath == "/content/gdrive/My Drive/Datasets/Augen/1/open-input-1" or
                        dirpath == "/content/gdrive/My Drive/Datasets/Augen/0/open-input-0"):
                        labels.append(2)
    return(images, labels)

def get_model():
    """
    Returns a compiled convolutional neural network model. Assume that the
    `input_shape` of the first layer is `(IMG_WIDTH, IMG_HEIGHT, 3)`.
    The output layer should have `NUM_CATEGORIES` units, one for each category.
    """
    INPUT_SHAPE = (IMG_WIDTH, IMG_HEIGHT, 3)

    #makes the network more robust
    data_augmentation = keras.Sequential(
      [
        layers.experimental.preprocessing.RandomFlip("horizontal", 
                                                    input_shape=(img_height, 
                                                                  img_width,
                                                                  3)),
        layers.experimental.preprocessing.RandomRotation(0.1),
        layers.experimental.preprocessing.RandomZoom(0.1),
      ]
    )
    # bias
    #model with several layers
    model = tf.keras.models.Sequential([
        data_augmentation,
        # add the convolution layer -> extract feature such as strong colour difference
        #-> lernt Kanten
        tf.keras.layers.Conv2D(64, (3, 3), activation="relu",
                               input_shape=INPUT_SHAPE),
        # use pooling -> verkleinert das Dild
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        # add another concolutional layer
        tf.keras.layers.Conv2D(128, (3, 3), activation="relu"),
        # pool again
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        #new
        tf.keras.layers.Conv2D(128, (3, 3), activation="relu"),#
        # generate a form as an input layer
        tf.keras.layers.Flatten(),
        # dense all are connected -> smaller then the imput layer
        tf.keras.layers.Dense(HIDDENLAYERS_SIZE, activation='relu'),
        # dense all are connected -> smaller then the imput layer
        tf.keras.layers.Dense(HIDDENLAYERS_SIZE, activation='relu'),
        # add a dropout
        tf.keras.layers.Dropout(DROPOUT),
        # add the output layer -> softmax gives a probability distribution min loss = 0 max = infinity
        # continues until the end
        tf.keras.layers.Dense(NUM_CATEGORIES, "softmax")
    ])
    # compile the model
    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy", 
        #metrics=["accuracy"]
        metrics=[tf.keras.metrics.Precision()]
    )
    return model

if __name__ == "__main__":
  main()

In [ ]:
import cv2
import numpy as np
import os
import sys
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import regularizers

from sklearn.model_selection import train_test_split

EPOCHS = 30
IMG_WIDTH = 30
IMG_HEIGHT = 30
NUM_CATEGORIES = 43
TEST_SIZE = 0.4
DROPOUT = 0.5
NORMALIZE = True
HIDDENLAYERS = 9
HIDDENLAYERS_SIZE = 256
PRETRAINED_CLASSIFIER = False
OPTIMIZER = True
STEPS_PER_EPOCH = 1


def main():

    # Check command-line arguments
    if len(sys.argv) not in [2, 3]:
        sys.exit("Usage: python traffic.py data_directory [model.h5]")

    # Get image arrays and labels for all image files
    images, labels = load_data(sys.argv[1])

    # Split data into training and testing sets
    labels = tf.keras.utils.to_categorical(labels)
    x_train, x_test, y_train, y_test = train_test_split(
        np.array(images), np.array(labels), test_size=TEST_SIZE
    )
    # Get a compiled neural network
    model = get_model()
    # Fit model on training data
    model.fit(x_train, y_train, epochs=EPOCHS)

    # Evaluate neural network performance
    model.evaluate(x_test,  y_test, verbose=2)

    # Save model to file
    if len(sys.argv) == 3:
        filename = sys.argv[2]
        model.save(filename)
        print(f"Model saved to {filename}.")


def load_data(data_dir):
    """
    Load image data from directory `data_dir`.

    Assume `data_dir` has one directory named after each category, numbered
    0 through NUM_CATEGORIES - 1. Inside each category directory will be some
    number of image files.

    Return tuple `(images, labels)`. `images` should be a list of all
    of the images in the data directory, where each image is formatted as a
    numpy ndarray with dimensions IMG_WIDTH x IMG_HEIGHT x 3. `labels` should
    be a list of integer labels, representing the categories for each of the
    corresponding `images`.
    """
    images = list()
    labels = list()
    # get currend directory
    workingDirectory = os.getcwd()
    # add them together
    path = workingDirectory + os.sep + data_dir
    # loop over all the directories
    for dirpath, dirnames, filenames in os.walk(path):
        if(filenames != ['.DS_Store']):
            # loops over all the files inside the directory
            for curFile in filenames:
                file = dirpath + os.sep + curFile
                # reads the image
                image = cv2.imread(file)
                if not PRETRAINED_CLASSIFIER:
                    # rezeises the image
                    resized = cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT),
                                         interpolation=cv2.INTER_AREA)
                else:
                    # size should be at least 32 for the other layers
                    # did not get this to work
                    # rezeises the image
                    resized = cv2.resize(image, (IMG_WIDTH + 2,
                                                 IMG_HEIGHT + 2),
                                         interpolation=cv2.INTER_AREA)
                # noramlize
                if(NORMALIZE):
                    resized = resized / 255.0
                # add to pictures
                images.append(resized)
                # add to labels
                labels.append(dirpath.replace(path + os.sep, ""))
    return(images, labels)


def get_model():
    """
    Returns a compiled convolutional neural network model. Assume that the
    `input_shape` of the first layer is `(IMG_WIDTH, IMG_HEIGHT, 3)`.
    The output layer should have `NUM_CATEGORIES` units, one for each category.
    """
    """
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation="relu",
                               input_shape=(30, 30, 3)),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(3, "softmax")
    ])
    """
    # decrease the rate of change towards the end
    lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
        0.001,
        decay_steps=STEPS_PER_EPOCH*1000,
        decay_rate=1,
        staircase=False)

    def get_optimizer():
        if OPTIMIZER:
            return tf.keras.optimizers.Adam(lr_schedule)
        # without an optimizer choose adam
        else:
            return "adam"
    # doesn't work
    if PRETRAINED_CLASSIFIER:
        INPUT_SHAPE = (IMG_WIDTH + 2, IMG_HEIGHT + 2, 3)
        # defining a base model
        base_model = tf.keras.applications.MobileNetV2(input_shape=INPUT_SHAPE,
                                                       include_top=False,
                                                       weights='imagenet')
        base_model.trainable = False
        global_average_layer = tf.keras.layers.GlobalAveragePooling2D
        model = tf.keras.Sequential([
            base_model,
            global_average_layer,
            tf.keras.layers.Dense(NUM_CATEGORIES, "softmax")
        ])
        model.compile(
            optimizer=get_optimizer(),
            loss="categorical_crossentropy",
            metrics=["accuracy"]
        )
        return model
    else:
        INPUT_SHAPE = (IMG_WIDTH, IMG_HEIGHT, 3)
        # bias
        model = tf.keras.models.Sequential([
            tf.keras.layers.Conv2D(64, (3, 3), activation="relu",
                                   input_shape=INPUT_SHAPE),
            tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
            tf.keras.layers.Conv2D(128, (3, 3), activation="relu"),
            tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
            tf.keras.layers.Flatten(),
            # dense all are connected -> smaller then the imput layer
            tf.keras.layers.Dense(
                HIDDENLAYERS_SIZE, kernel_regularizer=regularizers.l2(0.001),
                activation='relu'),
            tf.keras.layers.Dropout(DROPOUT),
            tf.keras.layers.Dense(NUM_CATEGORIES, "softmax")
        ])
        model.compile(
            optimizer="adam",
            loss="categorical_crossentropy",
            metrics=["accuracy"]
        )
        return model


if __name__ == "__main__":
    main()


/Deep_CNN
assets	keras_metadata.pb  saved_model.pb  variables


In [ ]:
import cv2
import numpy as np
import os
import sys
import tensorflow as tf

from sklearn.model_selection import train_test_split

# number of learnings should be until the loss converges otherwise we overfit
EPOCHS = 20
IMG_WIDTH = 30
IMG_HEIGHT = 30
NUM_CATEGORIES = 3
# number of images getting used for testing not for training in percent
TEST_SIZE = 0.4
# drop out nodes at random to make the network more robust
DROPOUT = 0.5
NORMALIZE = True
HIDDENLAYERS_SIZE = 128
# batchsize = x -> could define one // woundn't use it except if it takes to long
# it is the amount of images in the data set which is getting looked at before an adjustment


def main():
    # Get image arrays and labels for all image files
    images, labels = load_data(os.getcwd())
    print(labels)
    # Split data into training and testing sets
    labels = tf.keras.utils.to_categorical(labels)
    x_train, x_test, y_train, y_test = train_test_split(
        np.array(images), np.array(labels), test_size=TEST_SIZE
    )
    # Get a compiled neural network
    model = get_model()
    # Fit model on training data
    model.fit(x_train, y_train, epochs=EPOCHS)

    # Evaluate neural network performance
    model.evaluate(x_test,  y_test, verbose=2)

    # Save model to file
    if len(sys.argv) == 3:
        filename = sys.argv[2]
        #model.save(filename)
        #print(f"Model saved to {filename}.")

def load_data(data_dir):
    """
    Load image data from directory `data_dir`.

    Assume `data_dir` has one directory named after each category, numbered
    0 through NUM_CATEGORIES - 1. Inside each category directory will be some
    number of image files.

    Return tuple `(images, labels)`. `images` should be a list of all
    of the images in the data directory, where each image is formatted as a
    numpy ndarray with dimensions IMG_WIDTH x IMG_HEIGHT x 3. `labels` should
    be a list of integer labels, representing the categories for each of the
    corresponding `images`.
    """
    images = list()
    labels = list()

    # get currend directory
    #workingDirectory = os.getcwd()

    # add them together
    #path = workingDirectory + os.sep + data_dir
    path = data_dir
    # loop over all the directories
    for dirpath, dirnames, filenames in os.walk(path):
        print("x")
        if(filenames != ['.DS_Store']):
            # loops over all the files inside the directory
            for curFile in filenames:
                if(curFile != '.DS_Store' and curFile != 'a'):
                    file = dirpath + os.sep + curFile
                    # reads the image
                    image = cv2.imread(file)
                    # rezeises the image
                    resized = cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT),
                                        interpolation=cv2.INTER_AREA)
                    # make the input better for the neural network
                    if(NORMALIZE):
                        resized = resized / 255.0
                    # add to pictures
                    images.append(resized)
                    # add to labels
                    print("k")
                    if (dirpath == "/content/gdrive/MyDrive/Datasets/closed-input"):
                        print("!")
                        labels.append(0)
                    if (dirpath == "/content/gdrive/MyDrive/Datasets/no_face-input"):
                        labels.append(1)
                    if (dirpath == "/content/gdrive/MyDrive/Datasets/open-input"):
                        labels.append(2)
    return(images, labels)

def get_model():
    """
    Returns a compiled convolutional neural network model. Assume that the
    `input_shape` of the first layer is `(IMG_WIDTH, IMG_HEIGHT, 3)`.
    The output layer should have `NUM_CATEGORIES` units, one for each category.
    """
    INPUT_SHAPE = (IMG_WIDTH, IMG_HEIGHT, 3)
    # bias
    model = tf.keras.models.Sequential([
        # add the convolution layer
        tf.keras.layers.Conv2D(64, (3, 3), activation="relu",
                               input_shape=INPUT_SHAPE),
        # use pooling
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        # add another concolutional layer
        tf.keras.layers.Conv2D(128, (3, 3), activation="relu"),
        # pool again
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        # generate a form as an input layer
        tf.keras.layers.Flatten(),
        # dense all are connected -> smaller then the imput layer
        tf.keras.layers.Dense(HIDDENLAYERS_SIZE, activation='relu'),
        # add a dropout
        tf.keras.layers.Dropout(DROPOUT),
        # add the output layer
        tf.keras.layers.Dense(NUM_CATEGORIES, "softmax")
    ])
    # compile the model
    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model

if __name__ == "__main__":
  main()

x
[]


ValueError: ignored

In [ ]:
a = list()
a.append(1)
print(a)

[1]


In [ ]:
import cv2
import numpy as np
import os
import sys
import tensorflow as tf

from sklearn.model_selection import train_test_split

# number of learnings should be until the loss converges otherwise we overfit
EPOCHS = 10
IMG_WIDTH = 30
IMG_HEIGHT = 30
NUM_CATEGORIES = 43
# number of images getting used for testing not for training in percent
TEST_SIZE = 0.4
# drop out nodes at random to make the network more robust
DROPOUT = 0.5
NORMALIZE = True
HIDDENLAYERS_SIZE = 512
# batchsize = x -> could define one // woundn't use it except if it takes to long
# it is the amount of images in the data set which is getting looked at before an adjustment


def main():
    """
    old
    # Check command-line arguments
    #if len(sys.argv) not in [2, 3]:
    #    sys.exit("Usage: python traffic.py data_directory [model.h5]")
    """

    # Get image arrays and labels for all image files
    images, labels = load_data(sys.argv[1])

    # Split data into training and testing sets
    labels = tf.keras.utils.to_categorical(labels)
    x_train, x_test, y_train, y_test = train_test_split(
        np.array(images), np.array(labels), test_size=TEST_SIZE
    )
    # Get a compiled neural network
    model = get_model()
    # Fit model on training data
    model.fit(x_train, y_train, epochs=EPOCHS)

    # Evaluate neural network performance
    model.evaluate(x_test,  y_test, verbose=2)

    # Save model to file
    if len(sys.argv) == 3:
        filename = sys.argv[2]
        model.save(filename)
        print(f"Model saved to {filename}.")


def load_data(data_dir):
    """
    Load image data from directory `data_dir`.

    Assume `data_dir` has one directory named after each category, numbered
    0 through NUM_CATEGORIES - 1. Inside each category directory will be some
    number of image files.

    Return tuple `(images, labels)`. `images` should be a list of all
    of the images in the data directory, where each image is formatted as a
    numpy ndarray with dimensions IMG_WIDTH x IMG_HEIGHT x 3. `labels` should
    be a list of integer labels, representing the categories for each of the
    corresponding `images`.
    """
    images = list()
    labels = list()
    # get currend directory
    workingDirectory = os.getcwd()
    # add them together
    path = workingDirectory + os.sep + data_dir
    print(path)
    # loop over all the directories
    for dirpath, dirnames, filenames in os.walk(path):
        if(filenames != ['.DS_Store']):
            # loops over all the files inside the directory
            for curFile in filenames:
                file = dirpath + os.sep + curFile
                # reads the image
                image = cv2.imread(file)
                # rezeises the image
                resized = cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT),
                                     interpolation=cv2.INTER_AREA)
                # make the input better for the neural network
                if(NORMALIZE):
                    resized = resized / 255.0
                # add to pictures
                images.append(resized)
                # add to labels
                labels.append(dirpath.replace(path + os.sep, ""))
    return(images, labels)


def get_model():
    """
    Returns a compiled convolutional neural network model. Assume that the
    `input_shape` of the first layer is `(IMG_WIDTH, IMG_HEIGHT, 3)`.
    The output layer should have `NUM_CATEGORIES` units, one for each category.
    """
    INPUT_SHAPE = (IMG_WIDTH, IMG_HEIGHT, 3)
    # bias
    model = tf.keras.models.Sequential([
        # add the convolution layer
        tf.keras.layers.Conv2D(64, (3, 3), activation="relu",
                               input_shape=INPUT_SHAPE),
        # use pooling
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        # add another concolutional layer
        tf.keras.layers.Conv2D(128, (3, 3), activation="relu"),
        # pool again
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        # generate a form as an input layer
        tf.keras.layers.Flatten(),
        # dense all are connected -> smaller then the imput layer
        tf.keras.layers.Dense(HIDDENLAYERS_SIZE, activation='relu'),
        # add a dropout
        tf.keras.layers.Dropout(DROPOUT),
        # add the output layer
        tf.keras.layers.Dense(NUM_CATEGORIES, "softmax")
    ])
    # compile the model
    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model


if __name__ == "__main__":
    main()

In [ ]:
%cd /content/gdrive/MyDrive/Deep_CNN
!ls

/content/gdrive/MyDrive/Deep_CNN


In [ ]:
import cv2
import numpy as np
import os
import sys
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import regularizers
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

from sklearn.model_selection import train_test_split

# number of learnings should be until the loss converges otherwise we overfit
EPOCHS = 10
TOTAL_EPOCHS = EPOCHS + 20
IMG_WIDTH = 96
IMG_HEIGHT = 96
NUM_CATEGORIES = 3
# number of images getting used for testing not for training in percent
TEST_SIZE = 0.2
# drop out nodes at random to make the network more robust
DROPOUT = 0.9
NORMALIZE = True
HIDDENLAYERS_SIZE = 64
STEPS_PER_EPOCH = 1
IMG_SIZE = (IMG_WIDTH, IMG_HEIGHT)
IMG_SHAPE = (IMG_HEIGHT,IMG_WIDTH,3)
INPUT_SHAPE = (IMG_WIDTH, IMG_HEIGHT, 3)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                                  include_top=False,
                                               weights='imagenet')
LEARNING_RATE = 0.001
# batchsize = x -> could define one // woundn't use it except if it takes to long
# it is the amount of images in the data set which is getting looked at before an adjustment


def main():
    base_model.summary()
    # Get image arrays and labels for all image files
    images, labels = load_data(os.getcwd())
    print(labels)
    # Split data into training and testing sets
    labels = tf.keras.utils.to_categorical(labels)
    x_train, x_test, y_train, y_test = train_test_split(
        np.array(images), np.array(labels), test_size=TEST_SIZE
    )
    model = KerasClassifier(build_fn = lambda:get_model())
    # Fit model on training data
    #model.fit(x_train, y_train, epochs=EPOCHS)
    
    batch_size = [10, 50, 100]
    epochs = [5]
    dropout_rate = [0.1, 0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
    pool_type =  ['max', 'average']
    param_grid = dict(batch_size = batch_size, epochs = epochs)
    grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, cv=3,
                        verbose = 2)
    grid_result = grid.fit(x_train, y_train)
    # summarize results
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))
    # Evaluate neural network performance


    # Fit model on training data
    History = model.fit(x_train, y_train, epochs=EPOCHS)
    model.evaluate(x_test,  y_test, verbose=1)
    print("here")

    base_model.trainable = True

    # Let's take a look to see how many layers are in the base model
    print("Number of layers in the base model: ", len(base_model.layers))

    # Fine-tune from this layer onwards
    fine_tune_at = 100

    # Freeze all the layers before the `fine_tune_at` layer
    for layer in base_model.layers[:fine_tune_at]:
      layer.trainable =  False
    model.summary()

    for layer in base_model.layers[fine_tune_at:]:
      if layer.name[-2:] == "BN" or layer.name[-2:] == "bn":
        layer.trainable = False
    model.summary()
    
    decayrate = 1
    
    #finds the last learning rate of the pretrained model
    base_learning_rate = LEARNING_RATE / (1 + decayrate * STEPS_PER_EPOCH / STEPS_PER_EPOCH*1000)
    print(base_learning_rate)

    model.compile(loss="categorical_crossentropy",
              optimizer = tf.keras.optimizers.RMSprop(learning_rate=base_learning_rate),
              metrics="accuracy")

    history_fine =  model.fit(x_train, y_train, 
                         epochs=TOTAL_EPOCHS,
                         initial_epoch=History.epoch[-1])

    # Evaluate neural network performance
    model.evaluate(x_test,  y_test, verbose=1)
    """
    %cd ..
    %cd ..
    !ls 
    filename = ("/content/gdrive/MyDrive/Deep_CNN")
    !ls
    model.save(filename)
    %cd Datasets/Lachen
    """

    # Save model to file
    #if len(sys.argv) == 3:
    #    filename = sys.argv[2]
    #    model.save(filename)
    #    #print(f"Model saved to {filename}.")

def load_data(data_dir):
    """
    Load image data from directory `data_dir`.

    Assume `data_dir` has one directory named after each category, numbered
    0 through NUM_CATEGORIES - 1. Inside each category directory will be some
    number of image files.

    Return tuple `(images, labels)`. `images` should be a list of all
    of the images in the data directory, where each image is formatted as a
    numpy ndarray with dimensions IMG_WIDTH x IMG_HEIGHT x 3. `labels` should
    be a list of integer labels, representing the categories for each of the
    corresponding `images`.
    """
    images = list()
    labels = list()

    # get currend directory
    #workingDirectory = os.getcwd()

    # add them together
    #path = workingDirectory + os.sep + data_dir
    path = data_dir
    # loop over all the directories
    for dirpath, dirnames, filenames in os.walk(path):
        print("q")
        if(filenames != ['.DS_Store']):
            # loops over all the files inside the directory
            for curFile in filenames:
                if(curFile != '.DS_Store' and curFile != 'a'):
                    file = dirpath + os.sep + curFile
                    # reads the image
                    image = cv2.imread(file)
                    # rezeises the image
                    resized = cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT),
                                        interpolation=cv2.INTER_AREA)
                    # make the input better for the neural network
                    if(NORMALIZE):
                        resized = resized / 255.0
                    # add to pictures
                    images.append(resized)
                    # add to labels
                    if (dirpath == "/content/gdrive/My Drive/Datasets/Augen/3/closed-input-3" or 
                        dirpath == "/content/gdrive/My Drive/Datasets/Augen/2/closed-input-2" or
                        dirpath == "/content/gdrive/My Drive/Datasets/Augen/1/closed-input-1" or
                        dirpath == "/content/gdrive/My Drive/Datasets/Augen/0/closed-input-0"):
                        labels.append(0)
                        print(dirpath)
                    if (dirpath == "/content/gdrive/My Drive/Datasets/Augen/1/no_face-input-1" or
                        dirpath == "/content/gdrive/My Drive/Datasets/Augen/2/no_face-input-2" or
                        dirpath == "/content/gdrive/My Drive/Datasets/Augen/3/no_face-input-3" or 
                        dirpath == "/content/gdrive/My Drive/Datasets/Augen/0/no_face-input-0"):
                        labels.append(1)
                        print(dirpath)
                    if (dirpath == "/content/gdrive/My Drive/Datasets/Augen/3/open-input-3" or
                        dirpath == "/content/gdrive/My Drive/Datasets/Augen/2/open-input-2" or
                        dirpath == "/content/gdrive/My Drive/Datasets/Augen/1/open-input-1" or
                        dirpath == "/content/gdrive/My Drive/Datasets/Augen/0/open-input-0"):
                        labels.append(2)
                        print(dirpath)
    return(images, labels)

def get_model():
    """
    Returns a compiled convolutional neural network model. Assume that the
    `input_shape` of the first layer is `(IMG_WIDTH, IMG_HEIGHT, 3)`.
    The output layer should have `NUM_CATEGORIES` units, one for each category.
    """
    data_augmentation = tf.keras.Sequential(
      [
        tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal", 
                                                    input_shape=(IMG_HEIGHT, 
                                                                  IMG_WIDTH,
                                                                  3)),
        tf.keras.layers.experimental.preprocessing.RandomRotation(0.1),
        tf.keras.layers.experimental.preprocessing.RandomZoom(0.1),
      ]
    )

    # decrease the rate of change towards the end
    lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
        LEARNING_RATE,
        decay_steps=STEPS_PER_EPOCH*1000,
        decay_rate=1,
        staircase=False)
    
    # Create the base model from the pre-trained model MobileNet V2
    IMG_SHAPE = IMG_SIZE + (3,)
    base_model.trainable = False
    # bias
    #model with several layers
    model = tf.keras.models.Sequential([
        data_augmentation,                                
        # add the convolution layer -> extract feature such as strong colour difference
        #-> lernt Kanten
        base_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        # add a dropout
        tf.keras.layers.Dropout(DROPOUT),
        # add the output layer -> softmax gives a probability distribution min loss = 0 max = infinity
        # continues until the end
        tf.keras.layers.Dense(NUM_CATEGORIES, "softmax")
    ])
    # compile the model
    model.compile(
        optimizer=tf.keras.optimizers.Adam(lr_schedule),
        loss="categorical_crossentropy",
        metrics=["accuracy"]
        #metrics=[tf.keras.metrics.Precision()]
    )
    return model

if __name__ == "__main__":
  main()

Streaming output truncated to the last 5000 lines.
/content/gdrive/My Drive/Datasets/Augen/0/open-input-0
/content/gdrive/My Drive/Datasets/Augen/0/open-input-0
/content/gdrive/My Drive/Datasets/Augen/0/open-input-0
/content/gdrive/My Drive/Datasets/Augen/0/open-input-0
/content/gdrive/My Drive/Datasets/Augen/0/open-input-0
/content/gdrive/My Drive/Datasets/Augen/0/open-input-0
/content/gdrive/My Drive/Datasets/Augen/0/open-input-0
/content/gdrive/My Drive/Datasets/Augen/0/open-input-0
/content/gdrive/My Drive/Datasets/Augen/0/open-input-0
/content/gdrive/My Drive/Datasets/Augen/0/open-input-0
/content/gdrive/My Drive/Datasets/Augen/0/open-input-0
/content/gdrive/My Drive/Datasets/Augen/0/open-input-0
/content/gdrive/My Drive/Datasets/Augen/0/open-input-0
/content/gdrive/My Drive/Datasets/Augen/0/open-input-0
/content/gdrive/My Drive/Datasets/Augen/0/open-input-0
/content/gdrive/My Drive/Datasets/Augen/0/open-input-0
/content/gdrive/My Drive/Datasets/Augen/0/open-input-0
/content/gdriv

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Epoch 1/5
